In [1]:
import gzip
import pickle
import time

import pandas as pd
import numpy as np

import sklearn.metrics
import sklearn.utils
import sklearn.linear_model

from sklearn.cross_validation import LabelKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

/Users/aakash.gupta/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Path to people.csv from ReadHatKaggle data set
FEATURE_FILE ='Data/act_train_features.csv'
# Path to act_train.csv from RedHatKaggle data set
OUTPUT ='Data/act_train_output.csv'
# Path to the test file
TEST_FILE = 'Data/act_test_features.csv'

# Path to the pickle files
SAVE_AS_DIR = 'Data/pickle'

TEST_PREPROCESSED_FILE='Data/pickle/test_merged_without17304.pkl'
TRAIN_PREPROCESSED_FILE='Data/pickle/train_merged_without17304.pkl'
train_with_dates ='Data/pickle/train_withDateFeatures_without17304.pkl'
test_with_dates ='Data/pickle/test_withDateFeatures_without17304.pkl'
train_interpolated = 'Data/pickle/train_withInterpolations.pkl'
test_interpolated = 'Data/pickle/test_withleak.pkl'

In [3]:
test_data_df = pd.read_pickle(test_interpolated)
train_data_df = pd.read_pickle(train_interpolated)

IOError: [Errno 2] No such file or directory: 'Data/pickle/test_withleak.pkl'

In [4]:
test_data_d = pd.read_pickle(test_with_dates)

In [5]:
train_data_d = pd.read_pickle(train_with_dates)

In [6]:
Extra_columns = list(set(train_data_d.columns).difference(set(train_data_df.columns)))

In [7]:
Extra_columns

['pdays_till_activity',
 'adays_till_activity',
 'people_per_group',
 'next_outcome',
 'people_per_group_date',
 'prev_outcome',
 'diff_date',
 'activities_per_group_date',
 'worked_for_day']

In [8]:
print train_data_d.shape, test_data_d.shape
train_data_df = pd.merge(train_data_df,train_data_d[['activity_id']+Extra_columns],on='activity_id',how='left')
test_data_df = pd.merge(test_data_df,test_data_d[['activity_id']+Extra_columns],on='activity_id',how='left')

(1563770, 81) (333083, 81)


In [9]:
# print df.shape
print train_data_df.shape, test_data_df.shape
# print df.columns.values

(1398166, 81) (498687, 81)


In [10]:
is_dup = train_data_df.drop('activity_id',axis=1).duplicated()
df = train_data_df[~is_dup]

In [11]:
df = pd.concat([df,test_data_df])

In [12]:
print df.shape

(1317664, 81)


In [13]:
# #People per group
# ppp = df.groupby('people_group_1')['people_id'].apply(lambda x: len(x))
# ppp = ppp.reset_index()
# ppp.columns = ['people_group_1', 'people_per_group']
# df = pd.merge(df, ppp, on='people_group_1', how='left')

In [14]:
#Activities per group
app = df.groupby('people_group_1')['activity_id'].apply(lambda x: len(x))
app = app.reset_index()
app.columns = ['people_group_1', 'activities_per_group']
df = pd.merge(df, app, on='people_group_1', how='left')

In [15]:
# df.rename(columns={'activities_per_group_y':'activities_per_group'},inplace=True)

In [16]:
mindate = pd.Timestamp('2022-07-17 00:00:00')
maxdate = pd.Timestamp('2023-08-31 00:00:00')
minpdate = pd.Timestamp('2020-05-18 00:00:00')

# mindate = min(data['date'])
# maxdate = max(data['date'])
# minpdate = min(data['pdate'])

In [17]:
# people_date_mean 
pdm = df.groupby('people_group_1')['pdays_till_activity'].apply(lambda x: x.mean())
pdm = pdm.reset_index()
pdm.columns = ['people_group_1', 'people_date_mean']
df = pd.merge(df, pdm, on = 'people_group_1', how='left')

In [18]:
# df.rename(columns={'people_date_mean_y':'people_date_mean'},inplace=True)

In [19]:
# people_date_first
pdf = df.groupby('people_group_1')['pdays_till_activity'].apply(lambda x: x.min())
pdf = pdf.reset_index()
pdf.columns = ['people_group_1', 'people_date_first']
df = pd.merge(df, pdf, on = 'people_group_1', how='left')
# df.rename(columns={'people_date_first_y':'people_date_first'},inplace=True)

In [20]:
# activity_date_first
adf = df.groupby('people_group_1')['adays_till_activity'].apply(lambda x: x.min())
adf = adf.reset_index()
adf.columns = ['people_group_1', 'activity_date_first']
df = pd.merge(df, adf, on = 'people_group_1')
# df.rename(columns={'activity_date_first_y':'activity_date_first'},inplace=True)

In [21]:
# activity_date_last
adl = df.groupby('people_group_1')['adays_till_activity'].apply(lambda x: x.max())
adl = adl.reset_index()
adl.columns = ['people_group_1', 'activity_date_last']
df = pd.merge(df, adl, on = 'people_group_1')
# df.rename(columns={'activity_date_last_y':'activity_date_last'},inplace=True)

In [22]:
# people_date_last
pdl = df.groupby('people_group_1')['pdays_till_activity'].apply(lambda x: x.max())
pdl = pdl.reset_index()
pdl.columns = ['people_group_1', 'people_date_last']
df = pd.merge(df, pdl, on = 'people_group_1')
# df.rename(columns={'people_date_last_y':'people_date_last'},inplace=True)

In [23]:
# pchar_38_mean
p38m = df.groupby('people_group_1')['people_char_38'].apply(lambda x: x.mean())
p38m = p38m.reset_index()
p38m.columns = ['people_group_1', 'pchar_38_mean']
df = pd.merge(df, p38m, on = 'people_group_1', how='left')
# df.rename(columns={'pchar_38_mean_y':'pchar_38_mean'},inplace=True)

In [24]:
#pchar_38_median
p38med = df.groupby('people_group_1')['people_char_38'].apply(lambda x: x.median())
p38med = p38med.reset_index()
p38med.columns = ['people_group_1', 'pchar_38_median']
df = pd.merge(df, p38med, on = 'people_group_1',how='left')
# df.rename(columns={'pchar_38_median_y':'pchar_38_median'},inplace=True)

In [25]:
# num of activity dates
nad = df.groupby('people_group_1')['adays_till_activity'].apply(lambda x: len(x.unique()))
nad  = nad.reset_index()
nad.columns = ['people_group_1', 'num_activity_dates']
df = pd.merge(df, nad, on = 'people_group_1',how='left')
# df.rename(columns={'pchar_38_median_y':'pchar_38_median'},inplace=True)

In [26]:
set(df.columns.values).difference(set(train_data_df.columns.values))

{'activities_per_group',
 'activity_date_first',
 'activity_date_last',
 'num_activity_dates',
 'pchar_38_mean',
 'pchar_38_median',
 'people_date_first',
 'people_date_last',
 'people_date_mean'}

In [27]:
# for column in df.columns:
#     print (column+": " +str(len(df[column].unique())))

In [28]:
FEATURES = [
'people_group_1',
'people_per_group',
'activities_per_group',
'people_date_mean',
'people_date_first',
'activity_date_first',
'activity_date_last',
'people_date_last',
'pchar_38_mean',
'pchar_38_median',
'num_activity_dates',
'outcome']

In [29]:
# Tryi only with group related features first
df2 = df[FEATURES]
df2.shape

(1317664, 12)

In [30]:
df.to_pickle(SAVE_AS_DIR+'/train_test_group_columns.pkl')

In [31]:
# Keeping a copy for future use
tr = df
df = df2
print tr.shape,df.shape

(1317664, 90) (1317664, 12)


In [32]:
def group_ohe(df, key):

    val_unique = df[key].unique().astype(int).tolist()
    #create dataframe here

    df_encoded = pd.DataFrame(index = range(group_count), columns = ['people_group_1'] + val_unique)
    df_encoded = df_encoded.fillna(0.0)

    i=0
    for g in df.groupby('people_group_1'):
        df_encoded['people_group_1'][i] = g[0]
        
        group_size = len(g[1]) * 1.0
        
        vc = g[1][key].value_counts()

        for z in zip(vc.index.values, vc.values):
            df_encoded[z[0]][i] = z[1]/group_size
        i+=1
    
    
    wo_cols = ['people_group_1']
    for x in val_unique:
        wo_cols.append('{0}_group_onehotavg_{1}'.format(key, x))
    
    df_encoded.columns = wo_cols


    droplist = []
    for col in df_encoded.keys():
        if df_encoded[col].mean() < .001:
            droplist.append(col)


    for d in droplist:
        df_encoded.drop(d, axis=1, inplace=True)
    

    return df_encoded

In [33]:
lr = tr.copy()

In [34]:
oh_keys = []
alsouse = ['activity_category']
for k in tr.keys():
    if 'char_10' in k:
        continue
        
    if 'char_38' in k:
        continue
        
    if k in alsouse or 'char' in k:
        oh_keys.append(k)

In [36]:
group_count = len(tr['people_group_1'].unique())
for k in oh_keys:
    print(k)
    df_oh = group_ohe(tr, k)
    df = pd.merge(df, df_oh, on='people_group_1', how='left')

activity_category
char_1
char_2
char_3
char_4
char_5
char_6
char_7
char_8
char_9
people_char_1
people_char_11
people_char_12
people_char_13
people_char_14
people_char_15
people_char_16
people_char_17
people_char_18
people_char_19
people_char_2
people_char_20
people_char_21
people_char_22
people_char_23
people_char_24
people_char_25
people_char_26
people_char_27
people_char_28
people_char_29
people_char_3
people_char_30
people_char_31
people_char_32
people_char_33
people_char_34
people_char_35
people_char_36
people_char_37
people_char_4
people_char_5
people_char_6
people_char_7
people_char_8
people_char_9


In [37]:
for k in df.keys():
    print(k)

people_group_1
people_per_group
activities_per_group
people_date_mean
people_date_first
activity_date_first
activity_date_last
people_date_last
pchar_38_mean
pchar_38_median
num_activity_dates
outcome
activity_category_group_onehotavg_1
activity_category_group_onehotavg_2
activity_category_group_onehotavg_3
activity_category_group_onehotavg_4
activity_category_group_onehotavg_5
activity_category_group_onehotavg_6
activity_category_group_onehotavg_7
char_1_group_onehotavg_0
char_1_group_onehotavg_1
char_1_group_onehotavg_2
char_1_group_onehotavg_3
char_1_group_onehotavg_4
char_1_group_onehotavg_5
char_1_group_onehotavg_6
char_1_group_onehotavg_7
char_1_group_onehotavg_8
char_1_group_onehotavg_10
char_1_group_onehotavg_11
char_1_group_onehotavg_12
char_1_group_onehotavg_13
char_1_group_onehotavg_15
char_1_group_onehotavg_17
char_1_group_onehotavg_23
char_1_group_onehotavg_25
char_1_group_onehotavg_26
char_1_group_onehotavg_29
char_2_group_onehotavg_0
char_2_group_onehotavg_1
char_2_group

In [38]:
len(df.keys())

289

In [39]:
df.to_pickle(SAVE_AS_DIR+'/train_test_group_columns_adv.pkl')

In [40]:
train_unique_groups = df[~df['outcome'].isnull()]['people_group_1'].unique()
test_unique_groups = df[df['outcome'].isnull()]['people_group_1'].unique()


test_only_groups = []
for group in test_unique_groups:
    if group not in train_unique_groups:
        test_only_groups.append(group)

In [41]:
print len(train_unique_groups),len(test_unique_groups),len(test_only_groups)

29899 11639 4325


In [42]:
final_train_data_df = df[~df['outcome'].isnull()]
final_test_data_df = df[df['outcome'].isnull()]

print final_train_data_df.shape, final_test_data_df.shape

(984581, 289) (333083, 289)


In [46]:
train_data = df[~df['people_group_1'].isin(test_only_groups)]
test_data = df[df['people_group_1'].isin(test_only_groups)]

In [49]:
def getGroupOutputType(x):
    outcome = x.mean()
    if outcome == 1.0 :
        return 1
    if outcome == 0.0:
        return 0
    return 2

In [50]:
train_otype = train_data.groupby('people_group_1')['outcome'].apply(getGroupOutputType)
train_otype = train_otype.to_frame().reset_index()
train_data = pd.merge(train_data,train_otype,on='people_group_1',how='left')
train_data.shape

(1248591, 290)

In [51]:
train_data.rename(columns={'outcome_x':'outcome','outcome_y':'otype'},inplace=True)

In [52]:
is_dup = train_data.drop('outcome',axis=1).duplicated()
train_data = train_data[~is_dup]
train_data.shape

(29899, 290)

In [53]:
is_dup = test_data.duplicated()
test_data = test_data[~is_dup]
test_data.shape

(4325, 289)

In [54]:
def category_to_one_hot(dataset, non_feature, continuous_feature):
    # Function to change labels of categories to one-hot encoding using scikit's OneHot Encoding sparse matrix
    # pd.get_dummies(df) does the same, provides sweet header's as well but it kill's memory
    ds = dataset.drop(non_feature, axis=1)
    boolean_column = []
    counter = 0
    for column in ds.columns:
        if column not in continuous_feature:
            boolean_column.append(counter)
        counter += 1
    # boolean_column is not the column name but index
    print("Done filtering columns...")
    grd_enc = OneHotEncoder(categorical_features=boolean_column)
    encoded_arr = grd_enc.fit_transform(ds)
    return encoded_arr

In [55]:
train_data.to_pickle('group3d_train.pkl')

In [56]:
test_data.to_pickle('group3d_test.pkl')

In [54]:
# # Function to one hot encode all values ~ 120 secs
# start=time.time()
# trainarr=category_to_one_hot(train_data,NON_FEATURE+['otype'],CONT+CATEGORICAL_BINARY)
# end=time.time()
# print(end-start)

In [55]:
# start=time.time()
# testarr=category_to_one_hot(final_test_data_df,NON_FEATURE,CONT+CATEGORICAL_BINARY)
# end=time.time()
# print(end-start)

In [56]:
# print trainarr.shape
# print testarr.shape

In [144]:
# test_data['outcome']

In [57]:
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss
import xgboost as xgb

In [254]:
# trainarr = trainarr.tocsr()

In [157]:
# train_data[:6]

In [156]:
# train_data.iloc[5]

In [58]:
## Gives better distribution ##
from sklearn.model_selection import StratifiedKFold

KfoldOnPId=train_data

trainarr=KfoldOnPId.drop(['otype','outcome'],axis=1)
y = KfoldOnPId['otype'].values


X_train, X_test = {}, {}
y_train, y_test = {}, {}

test_indexes = []

skf = StratifiedKFold(5, shuffle=True, random_state=12345)
i=0
for train_index, test_index in skf.split(trainarr,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    test_indexes.append(test_index)
    X_train[i], X_test[i] = trainarr.iloc[train_index], trainarr.iloc[test_index]
    y_train[i], y_test[i] = y[train_index], y[test_index]
    i +=1

('TRAIN:', array([    0,     1,     2, ..., 29895, 29896, 29898]), 'TEST:', array([    3,     7,    11, ..., 29888, 29891, 29897]))
('TRAIN:', array([    0,     1,     2, ..., 29896, 29897, 29898]), 'TEST:', array([    4,    10,    19, ..., 29878, 29889, 29894]))
('TRAIN:', array([    0,     1,     2, ..., 29895, 29897, 29898]), 'TEST:', array([    5,    20,    28, ..., 29890, 29893, 29896]))
('TRAIN:', array([    0,     1,     3, ..., 29895, 29896, 29897]), 'TEST:', array([    2,    12,    14, ..., 29887, 29892, 29898]))
('TRAIN:', array([    2,     3,     4, ..., 29896, 29897, 29898]), 'TEST:', array([    0,     1,     6, ..., 29876, 29883, 29895]))


In [59]:
for k in range(len(X_train)):
    print(k, len(X_test[k]), y_test[k].mean(),len(X_train[k]), y_train[k].mean())

(0, 5981, 0.72094967396756393, 23918, 0.72092148172924153)
(1, 5981, 0.72094967396756393, 23918, 0.72092148172924153)
(2, 5980, 0.7210702341137124, 23919, 0.72089134161127133)
(3, 5979, 0.72085633049004849, 23920, 0.72094481605351168)
(4, 5978, 0.72080963532954168, 23921, 0.7209564817524351)


In [60]:
xg_train = {}
xg_test = {}

for f in range(5):
    print y_train[f].shape, y_test[f].shape
    xg_train[f] = xgb.DMatrix( X_train[f], label=y_train[f])
    xg_test[f] = xgb.DMatrix(X_test[f], label=y_test[f])

(23918,) (5981,)
(23918,) (5981,)
(23919,) (5980,)
(23920,) (5979,)
(23921,) (5978,)


In [61]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['eval_metric'] = 'mlogloss'

bst = {}
pred = {}
for f in range(5):
    
    watchlist = [ (xg_train[f],'train'), (xg_test[f], 'test') ]
    num_round = 2500
    bst[f] = xgb.train(param, xg_train[f], num_round, watchlist , verbose_eval = 10, early_stopping_rounds=30)
    # get prediction
#    pred[f] = bst.predict( xg_test[f] )
    

[0]	train-mlogloss:1.09499	test-mlogloss:1.09521
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 30 rounds.
[10]	train-mlogloss:1.06107	test-mlogloss:1.06368
[20]	train-mlogloss:1.03024	test-mlogloss:1.03534
[30]	train-mlogloss:1.00362	test-mlogloss:1.0109
[40]	train-mlogloss:0.9799	test-mlogloss:0.989373
[50]	train-mlogloss:0.958724	test-mlogloss:0.970189
[60]	train-mlogloss:0.939327	test-mlogloss:0.952801
[70]	train-mlogloss:0.922449	test-mlogloss:0.937826
[80]	train-mlogloss:0.906329	test-mlogloss:0.923652
[90]	train-mlogloss:0.892233	test-mlogloss:0.9115
[100]	train-mlogloss:0.87913	test-mlogloss:0.900292
[110]	train-mlogloss:0.867563	test-mlogloss:0.89049
[120]	train-mlogloss:0.856685	test-mlogloss:0.881456
[130]	train-mlogloss:0.846616	test-mlogloss:0.873237
[140]	train-mlogloss:0.837206	test-mlogloss:0.865583
[150]	train-mlogloss:0.828419	test-mlogloss:0.85858
[160]	train-mlogloss:0.82057

In [62]:
prev_train = train_data.copy()

In [63]:
from sklearn.metrics import log_loss
import sklearn

xgb_cv_pred = {}
train_data = prev_train.copy()
df_preds = []
for f in range(5):
    xgb_cv_pred[f] = bst[f].predict(xg_test[f], ntree_limit=bst[f].best_ntree_limit)
    # Append all DataFrames
    df_preds.append(pd.DataFrame(xgb_cv_pred[f]))
    df_preds[-1].columns = ['gp_all0', 'gp_all1', 'gp_mixed']
    df_preds[-1]['people_group_1'] = train_data.iloc[test_indexes[f]].people_group_1.values
    

df_preds_all = pd.concat(df_preds)

print ("-----------")
y_tots = []
y_prids = []
for l in range(5):
    y_tots.append(y_test[l])
    y_prids.append(xgb_cv_pred[l])

y_tots = np.concatenate(y_tots)
y_prids = np.concatenate(y_prids)

print (log_loss(y_tots,y_prids))
print (y_tots.shape)
print (y_prids.shape)
print ("-----------")
train_preds = np.array(df_preds_all.sort_values('people_group_1').drop('people_group_1', axis=1))
train_data.sort_values('people_group_1',inplace=True)
print (train_preds.shape)
print (train_data.shape)

log_loss(train_data.otype.values, train_preds)

-----------
0.745052387173
(29899,)
(29899, 3)
-----------
(29899, 3)
(29899, 290)


0.74505238717287336

In [64]:
df_preds_all[:5]

,gp_all0,gp_all1,gp_mixed,people_group_1
0,0.079944,0.715920,0.204135,20178
1,0.254505,0.402339,0.343156,38568
2,0.679744,0.128426,0.191830,17737
3,0.099004,0.633631,0.267365,4482
4,0.007132,0.849459,0.143409,1265


In [65]:
xg_test_r = xgb.DMatrix(test_data.drop('outcome',axis=1))

In [66]:
# Get for test
test_preds = []
for f in range(5):
    test_preds.append(bst[f].predict(xg_test_r, ntree_limit = bst[f].best_ntree_limit))
    
preds_comb = np.mean([test_preds[j] for j in range(1,5)], axis = 0)

df_test_preds = pd.DataFrame(preds_comb)
df_test_preds.columns = ['gp_all0', 'gp_all1', 'gp_mixed']

In [67]:
test_data['gp_all0'] = 0
test_data['gp_all1'] = 0
test_data['gp_mixed'] = 0

In [68]:
test_data[['gp_all0', 'gp_all1', 'gp_mixed']] = df_test_preds.as_matrix()

In [69]:
print (train_data.shape,df_preds_all.shape, test_data.shape)

((29899, 290), (29899, 4), (4325, 292))


In [70]:
train_data = pd.merge(train_data,df_preds_all,on='people_group_1',how='left')

In [71]:
final_df = pd.concat([test_data,train_data.drop('otype',axis=1)])

In [72]:
final_df.shape

(34224, 292)

In [73]:
new_df = pd.concat([train_data_df,test_data_df])

In [74]:
new_df.shape

(1896853, 81)

In [75]:
final_df_1 = pd.merge(new_df,final_df[['people_group_1','gp_all0', 'gp_all1', 'gp_mixed']],on='people_group_1',how='left')

In [76]:
final_train_data_df = final_df_1[~final_df_1['outcome'].isnull()]
final_test_data_df = final_df_1[final_df_1['outcome'].isnull()]

print final_train_data_df.shape, final_test_data_df.shape

(1563770, 84) (333083, 84)


In [77]:
print train_data_df.shape, test_data_df.shape

(1398166, 81) (498687, 81)


In [78]:
final_test_data_df.to_pickle(SAVE_AS_DIR+'/test2_otype.pkl')
final_train_data_df.to_pickle(SAVE_AS_DIR+'/train2_otype.pkl')

In [ ]:
############################################### END ################################################################################################